# Compare Radiomics extraction modes

1. extraction per lung
2. extraction per lung and stacked 
3. extraction combined (using one run)

In [49]:
import sys, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import auc, roc_curve, roc_auc_score, mean_absolute_error, balanced_accuracy_score, f1_score, accuracy_score, make_scorer, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline as Pipeline_sampler
from sklearn.model_selection import train_test_split

seed = 42

In [50]:
sys.path.append(os.path.abspath('src/'))
from data.lungdataset import LungData

rootdir = "/Volumes/Samsung_T5/MA/manifest-1641816674790/subsample_thresh1_A"
ld = LungData(rootdir)
medical_df = ld.raw_ehd

In [51]:
rad_1 = pd.read_csv("data/radiomics/thresh1_radiomics_all.csv")
rad_2 = pd.read_csv("data/radiomics/thresh1_radiomics_all_stacked.csv")
rad_3 = pd.read_csv("data/radiomics/thresh1_radiomics_all_combined.csv")

dfs = [rad_1, rad_2, rad_3]

rad_1 = ld.prepare_multiclass_for_radiomics(rad_1, medical_df, verbose=False)
rad_2 = ld.prepare_multiclass_for_radiomics(rad_2, medical_df, verbose=False)
rad_3 = ld.prepare_multiclass_for_radiomics(rad_3, medical_df, verbose=False)

_ids = ["id", "xray_path", "left", "y"]

print(rad_1.drop(_ids, axis=1).shape, rad_2.drop(_ids, axis=1, errors="ignore").shape, rad_3.drop(_ids, axis=1).shape)
rad_1.head()

(9456, 102) (4728, 204) (4728, 102)


,10Percentile,90Percentile,Energy,Entropy,InterquartileRange,Kurtosis,Maximum,MeanAbsoluteDeviation,Mean,Median,...,LargeDependenceHighGrayLevelEmphasis,LargeDependenceLowGrayLevelEmphasis,LowGrayLevelEmphasis,SmallDependenceEmphasis,SmallDependenceHighGrayLevelEmphasis,SmallDependenceLowGrayLevelEmphasis,id,xray_path,left,y
0,71.0,150.0,394369086.0,2.394008,41.0,3.095117,218.0,24.855771,113.581023,116.0,...,1202.128658,2.250114,0.052380,0.054211,1.557875,0.002838,A860070,/Volumes/Samsung_T5/MA/manifest-1641816674790/...,True,3
1,39.0,142.0,256883687.0,2.665426,53.0,2.753274,237.0,30.943247,94.441930,97.0,...,840.518399,4.847234,0.110090,0.056541,1.328324,0.005729,A860070,/Volumes/Samsung_T5/MA/manifest-1641816674790/...,False,3
2,55.0,149.0,296824858.0,2.539965,50.0,2.468845,194.0,28.485066,104.368473,107.0,...,1103.019745,3.437842,0.072981,0.048489,1.231922,0.003346,A860070,/Volumes/Samsung_T5/MA/manifest-1641816674790/...,True,3
3,47.0,154.0,377337361.0,2.685669,65.0,2.055343,203.0,34.010481,99.428671,98.0,...,937.161292,3.852431,0.087336,0.059224,1.406542,0.005033,A860070,/Volumes/Samsung_T5/MA/manifest-1641816674790/...,False,3
4,54.0,161.0,707815625.0,2.702847,57.0,2.444984,211.0,32.624710,111.785194,116.0,...,1192.222322,3.080281,0.071997,0.054359,1.554756,0.003835,A860070,/Volumes/Samsung_T5/MA/manifest-1641816674790/...,True,3


In [52]:
def drop_corr_columns(df, threshold=0.95):
    dfs = pd.DataFrame(StandardScaler().fit_transform(df), columns=df.columns)
    corr_thresh = threshold
    corr_matrix = dfs.corr().abs() # Create correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)) # Select upper triangle of correlation matrix
    to_drop = [column for column in upper.columns if any(upper[column] > corr_thresh)] # Find features with correlation greater than 0.95
    df = df.drop(to_drop, axis=1) # Drop features 
    print("Dropped {} features!".format(len(to_drop)))
    return df



In [68]:
dfs = [rad_1, rad_2, rad_3]
extraction_methods = ["Separated", "Stacked", "Combined"]

res = pd.DataFrame()

for df, method in zip(dfs, extraction_methods):
    X = df.drop(_ids, axis=1, errors="ignore")
    y = df["y"].copy()
    y[y==5] = 7

    X = drop_corr_columns(X)
    print("     X-shape: {}".format(X.shape))

    # split
    x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                        stratify=y, random_state=seed)

    # train
    pipe = Pipeline([   ("scaler", StandardScaler()),
                        ('svm', SVC(probability=True, class_weight="balanced", random_state=seed)),
                    ])
    
    scoring = { 'accuracy': make_scorer(accuracy_score),
                'balanced_accuracy': make_scorer(balanced_accuracy_score),
                #'f1_weighted': make_scorer(f1_score, average = 'weighted'),
                'roc_auc': make_scorer(roc_auc_score, multi_class="ovo", needs_proba=True, average="weighted"),
                'recall': make_scorer(recall_score, average = 'micro'),
            }
    grid = {}
    gscv = GridSearchCV(pipe, grid, scoring=scoring, n_jobs=4, cv=10, refit="roc_auc")
    gscv.fit(x_train, y_train)

    # deal with results
    _res = pd.DataFrame(gscv.cv_results_)
    _res.index=[method]
    res = pd.concat([res, _res])

    for score_name, scorer in scoring.items():
        res.at[method, f"val-{score_name}"] = scorer(gscv.best_estimator_, x_val, y_val)

Dropped 35 features!
     X-shape: (9456, 67)
Dropped 73 features!
     X-shape: (4728, 131)
Dropped 45 features!
     X-shape: (4728, 57)


In [69]:
res_table = res.filter(regex="mean_test").round(3)
_metrics = ["Accuracy", "Balanced-Accuracy", "ROC-AUC", "Recall"] 
res_table.columns=_metrics
res_std_table = res.filter(regex="std_test").round(3)
res_std_table.columns=_metrics
res_table_string = res_table[_metrics].astype(str) + "\n +/-" + res_std_table[_metrics].astype(str)
res_table_string

,Accuracy,Balanced-Accuracy,ROC-AUC,Recall
Separated,0.386\n +/-0.018,0.302\n +/-0.029,0.734\n +/-0.013,0.386\n +/-0.018
Stacked,0.456\n +/-0.022,0.326\n +/-0.056,0.74\n +/-0.033,0.456\n +/-0.022
Combined,0.428\n +/-0.035,0.325\n +/-0.045,0.759\n +/-0.025,0.428\n +/-0.035


In [73]:
res_val_table = res.filter(regex="val").round(3)
res_val_table.columns=_metrics
print(res_val_table.to_latex(caption="Results for the different methods of extracting radiomics features."))

\begin{table}
\centering
\caption{Results for the different methods of extracting radiomics features.}
\begin{tabular}{lrrrr}
\toprule
{} &  Accuracy &  Balanced-Accuracy &  ROC-AUC &  Recall \\
\midrule
Separated &     0.399 &              0.292 &    0.723 &   0.399 \\
Stacked   &     0.483 &              0.330 &    0.766 &   0.483 \\
Combined  &     0.436 &              0.341 &    0.754 &   0.436 \\
\bottomrule
\end{tabular}
\end{table}

